In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp -r drive/MyDrive/<folder_name>/* .

# create folder on drive and insert folder name above

In [3]:
import os
import torch
os.environ['TORCH'] = torch.__version__

!pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install pyg-lib -f https://data.pyg.org/whl/nightly/torch-${TORCH}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 66.5 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 43.1 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/nightly/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.2 MB/s eta 0:00:00
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-yksf2bg5
  Running command git clone --filter=blob:none --quiet https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-yksf2bg5
  Resolved https://github.com/pyg-team/pytorch_geometric.git to commit ec6264bd9cd39f8f47abcad1a899b53fa92ae6d6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.4.0-py3-none-any.whl

In [59]:
import torch
from torch import Tensor
import pandas as pd
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T
from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.nn import SAGEConv, to_hetero
import torch.nn.functional as F
import tqdm
from sklearn.model_selection import train_test_split
import random
import numpy as np

In [60]:
crs = []
cnt = []

with open('./gnn_rel/course_concepts.txt', 'r') as f:
    lines = [l.strip() for l in f.readlines()]
    for line in lines:
        c_cs = line.split('\t')
        if(len(c_cs) == 1):
            continue
        cs = c_cs[1].split(' ')
        c = [int(c_cs[0])] * len(cs)

        crs.extend(c)
        cnt.extend([int(css) for css in cs])

edge_index_course_to_concept = torch.stack([torch.tensor(crs, dtype=int), torch.tensor(cnt, dtype=int)])
edge_index_course_to_concept

tensor([[   0,    0,    0,  ...,  686,  686,  686],
        [2612,  495,  500,  ..., 2916, 2694, 2695]])

In [61]:
crs = []
sch = []

with open('./gnn_rel/course_school.txt', 'r') as f:
    lines = [l.strip() for l in f.readlines()]
    for line in lines:
        c_schools = line.split('\t')
        if(len(c_schools) == 1):
            continue
        schools = c_schools[1].split(' ')
        c = [int(c_schools[0])] * len(schools)

        crs.extend(c)
        sch.extend([int(s) for s in schools])


edge_index_course_to_school = torch.stack([torch.tensor(crs, dtype=int), torch.tensor(sch, dtype=int)])
edge_index_course_to_school

tensor([[  0,   1,   2,  ..., 684, 685, 686],
        [122, 122, 122,  ..., 147,   7,  11]])

In [62]:
crs = []
tch = []

with open('./gnn_rel/course_teachers.txt', 'r') as f:
    lines = [l.strip() for l in f.readlines()]
    for line in lines:
        c_teachers = line.split('\t')
        if(len(c_teachers) == 1):
            continue
        teachers = c_teachers[1].split(' ')
        c = [int(c_teachers[0])] * len(teachers)

        crs.extend(c)
        tch.extend([int(t) for t in teachers])


edge_index_course_to_teacher = torch.stack([torch.tensor(crs, dtype=int), torch.tensor(tch, dtype=int)])
edge_index_course_to_teacher

tensor([[   0,    0,    0,  ...,  686,  686,  686],
        [ 993, 1481, 1489,  ..., 1118, 1211, 1448]])

In [63]:
user_enrolments = {}

with open('./gnn_rel/enrolments.txt', 'r') as f:
    lines = [l.strip() for l in f.readlines()]
    for line in lines:
        user_course = line.split('\t')

        user = user_course[0]
        course = user_course[1]

        user_enrolments[user] = user_enrolments.get(user, []) + [course]

In [64]:
def create_data_file(path, data):
    with open(path, 'w') as f:
        f.writelines(data)

In [65]:
train_data = []
test_data = []
for user in user_enrolments:
    courses = user_enrolments[user]
    user_train_data, user_test_data = train_test_split(courses, train_size=0.8)

    for c in user_train_data:
        train_data.append(f"{user}\t{c}\n")

    for c in user_test_data:
        test_data.append(f"{user}\t{c}\n")

random.shuffle(train_data)
random.shuffle(test_data)

#create_data_file('./gnn_rel/train.txt', train_data)
#create_data_file('./gnn_rel/test.txt', test_data)

In [66]:
users_tmp = []
courses_tmp = []

with open('./gnn_rel/train.txt', 'r') as f:
    lines = [l.strip() for l in f.readlines()]
    for line in lines:
        user_course = line.split('\t')

        user = user_course[0]
        course = user_course[1]

        users_tmp.append(int(user))
        courses_tmp.append(int(course))


edge_index_user_to_course = torch.stack([torch.tensor(users_tmp, dtype=int), torch.tensor(courses_tmp, dtype=int)])
edge_index_user_to_course

tensor([[3424, 5806,  372,  ..., 5818, 2464,    2],
        [  29,   24,   67,  ...,  278,   69,   24]])

In [67]:
with open('./gnn_rel/users.txt', 'r') as f:
    users = [line.strip() for line in f.readlines()]
with open('./gnn_rel/courses.txt', 'r') as f:
    courses = [line.strip() for line in f.readlines()]
with open('./gnn_rel/schools.txt', 'r') as f:
    schools = [line.strip() for line in f.readlines()]
with open('./gnn_rel/teachers.txt', 'r') as f:
    teachers = [line.strip() for line in f.readlines()]
with open('./gnn_rel/concepts.txt', 'r') as f:
    concepts = [line.strip() for line in f.readlines()]

In [68]:
data = HeteroData()

EMBED_SIZE = 16

data['user'].node_id = torch.arange(len(users))
data['course'].node_id = torch.arange(len(courses))
data['school'].node_id = torch.arange(len(schools))
data['teacher'].node_id = torch.arange(len(teachers))
data['concept'].node_id = torch.arange(len(concepts))


# data['user'].x = torch.rand(size=(len(users), EMBED_SIZE))
# data['course'].x = torch.rand(size=(len(courses), EMBED_SIZE))
# data['school'].x = torch.rand(size=(len(schools), EMBED_SIZE))
# data['teacher'].x = torch.rand(size=(len(teachers), EMBED_SIZE))
# data['concept'].x = torch.rand(size=(len(concepts), EMBED_SIZE))


In [69]:
data['user', 'enroll', 'course'].edge_index = edge_index_user_to_course
data['course', 'course_school', 'school'].edge_index = edge_index_course_to_school
data['course', 'course_teacher', 'teacher'].edge_index = edge_index_course_to_teacher
data['course', 'course_concept', 'concept'].edge_index = edge_index_course_to_concept

In [70]:
data = T.ToUndirected()(data)

In [71]:
data

HeteroData(
  user={ node_id=[6548] },
  course={ node_id=[687] },
  school={ node_id=[150] },
  teacher={ node_id=[1683] },
  concept={ node_id=[3180] },
  (user, enroll, course)={ edge_index=[2, 75597] },
  (course, course_school, school)={ edge_index=[2, 687] },
  (course, course_teacher, teacher)={ edge_index=[2, 2284] },
  (course, course_concept, concept)={ edge_index=[2, 103497] },
  (course, rev_enroll, user)={ edge_index=[2, 75597] },
  (school, rev_course_school, course)={ edge_index=[2, 687] },
  (teacher, rev_course_teacher, course)={ edge_index=[2, 2284] },
  (concept, rev_course_concept, course)={ edge_index=[2, 103497] }
)

In [72]:
transform = T.RandomLinkSplit(
    num_val=0,
    num_test=0,
    disjoint_train_ratio=0.0,
    neg_sampling_ratio=2,
    add_negative_train_samples=False,
    edge_types=('user', 'enroll', 'course'),
    rev_edge_types=('course', 'rev_enroll', 'user')
)

train_data, val_data, test_data = transform(data)
#train_data = data

In [18]:
train_data

HeteroData(
  user={ node_id=[6548] },
  course={ node_id=[687] },
  school={ node_id=[150] },
  teacher={ node_id=[1683] },
  concept={ node_id=[3180] },
  (user, enroll, course)={
    edge_index=[2, 75597],
    edge_label=[75597],
    edge_label_index=[2, 75597],
  },
  (course, course_school, school)={ edge_index=[2, 687] },
  (course, course_teacher, teacher)={ edge_index=[2, 2284] },
  (course, course_concept, concept)={ edge_index=[2, 103497] },
  (course, rev_enroll, user)={ edge_index=[2, 75597] },
  (school, rev_course_school, course)={ edge_index=[2, 687] },
  (teacher, rev_course_teacher, course)={ edge_index=[2, 2284] },
  (concept, rev_course_concept, course)={ edge_index=[2, 103497] }
)

In [19]:
len(train_data['user', 'enroll', 'course'].edge_label),train_data['user', 'enroll', 'course'].edge_label.sum()

(75597, tensor(75597.))

In [73]:
edge_label_index = train_data['user', 'enroll', 'course'].edge_label_index
edge_label = train_data['user', 'enroll', 'course'].edge_label

In [21]:
train_loader = LinkNeighborLoader(
    data=train_data,
    num_neighbors={('course', 'course_school', 'school'):[2 ,2],
 ('course', 'course_teacher', 'teacher'):[2 ,2],
 ('course', 'course_concept', 'concept'):[7,7],
                   ('user', 'enroll', 'course'):[1,0],
                   ('course', 'rev_enroll', 'user'):[0,0],
 ('school', 'rev_course_school', 'course'):[2 ,2],
 ('teacher', 'rev_course_teacher', 'course'):[2 ,2],
 ('concept', 'rev_course_concept', 'course'):[7,7]},
    neg_sampling_ratio=2.0,
    edge_label_index=(('user', 'enroll', 'course'), edge_label_index),
    edge_label=edge_label,
    batch_size=128,
    shuffle=True
)

In [23]:
sampled_data = next(iter(train_loader))

In [ ]:
import networkx as nx
from matplotlib import pyplot as plt
from torch_geometric.nn import to_hetero
from torch_geometric.utils import to_networkx

hom = sampled_data.to_homogeneous()
g = to_networkx(hom)
# Networkx seems to create extra nodes from our heterogeneous graph, so I remove them
isolated_nodes = [node for node in g.nodes() if g.out_degree(node) == 0]
#[g.remove_node(i_n) for i_n in isolated_nodes]
# Plot the graph
fig, ax = plt.subplots(dpi=300, figsize=(20, 20))
nx.draw(g, pos=nx.spring_layout(g), with_labels=True, node_color=hom.node_type, ax =ax)
plt.show()

In [74]:
class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()

        self.conv1 = SAGEConv(hidden_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)

    def forward(self, x: Tensor, edge_index: Tensor):
        print(x)
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

class Classifier(torch.nn.Module):
    def forward(self, x_user: Tensor, x_course: Tensor, edge_label_index: Tensor) -> Tensor:
        edge_feat_user = x_user[edge_label_index[0]]
        edge_feat_course = x_course[edge_label_index[1]]

        return (edge_feat_user * edge_feat_course).sum(dim=-1)


class GNNSimple(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()

        self.conv1 = SAGEConv(hidden_channels, hidden_channels)

    def forward(self, x: Tensor, edge_index: Tensor):
        print(x)
        x = self.conv1(x, edge_index)
        return x


In [75]:
class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.user_emb = torch.nn.Embedding(data["user"].num_nodes, hidden_channels)
        self.course_emb = torch.nn.Embedding(data["course"].num_nodes, hidden_channels)
        self.concept_emb = torch.nn.Embedding(data["concept"].num_nodes, hidden_channels)
        self.school_emb = torch.nn.Embedding(data["school"].num_nodes, hidden_channels)
        self.teacher_emb = torch.nn.Embedding(data["teacher"].num_nodes, hidden_channels)

        # Instantiate homogeneous GNN:
        self.gnn = GNN(hidden_channels)

        # Convert GNN model into a heterogeneous variant:
        self.gnn = to_hetero(self.gnn, metadata=data.metadata())

        self.classifier = Classifier()

    def forward(self, data: HeteroData) -> Tensor:
        x_dict = {
          "user": self.user_emb(data["user"].node_id),
          "course": self.course_emb(data["course"].node_id),
          'concept': self.concept_emb(data['concept'].node_id),
          'school': self.school_emb(data['school'].node_id),
          'teacher': self.teacher_emb(data['teacher'].node_id)
        }

        # `x_dict` holds feature matrices of all node types
        # `edge_index_dict` holds all edge indices of all edge types
        x_dict = self.gnn(x_dict, data.edge_index_dict)

        pred = self.classifier(
            x_dict["user"],
            x_dict["course"],
            data["user", "enroll", "course"].edge_label_index,
        )

        return pred

In [76]:
class ModelSimple(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.user_emb = torch.nn.Embedding(data["user"].num_nodes, hidden_channels)
        self.course_emb = torch.nn.Embedding(data["course"].num_nodes, hidden_channels)
        self.concept_emb = torch.nn.Embedding(data["concept"].num_nodes, hidden_channels)
        self.school_emb = torch.nn.Embedding(data["school"].num_nodes, hidden_channels)
        self.teacher_emb = torch.nn.Embedding(data["teacher"].num_nodes, hidden_channels)

        # Instantiate homogeneous GNN:
        self.gnn = GNNSimple(hidden_channels)

        # Convert GNN model into a heterogeneous variant:
        self.gnn = to_hetero(self.gnn, metadata=data.metadata())

        self.classifier = Classifier()

    def forward(self, data: HeteroData) -> Tensor:
        x_dict = {
          "user": self.user_emb(data["user"].node_id),
          "course": self.course_emb(data["course"].node_id),
          'concept': self.concept_emb(data['concept'].node_id),
          'school': self.school_emb(data['school'].node_id),
          'teacher': self.teacher_emb(data['teacher'].node_id)
        }

        # `x_dict` holds feature matrices of all node types
        # `edge_index_dict` holds all edge indices of all edge types
        x_dict = self.gnn(x_dict, data.edge_index_dict)

        pred = self.classifier(
            x_dict["user"],
            x_dict["course"],
            data["user", "enroll", "course"].edge_label_index,
        )

        return pred

In [40]:
model = Model(hidden_channels=EMBED_SIZE)

Proxy(x)


In [77]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

#model = model.to(device)
#optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#num_epochs = 10

Device: cpu


In [84]:
def train(model, num_epochs, lr):
  model = model.to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=lr)
  model.train()


  for epoch in range(num_epochs):
      total_loss = total_examples = 0

      for sampled_data in tqdm.tqdm(train_loader):
          optimizer.zero_grad()
          sampled_data.to(device)
          pred = model(sampled_data)
          ground_truth = sampled_data['user', 'enroll', 'course'].edge_label
          loss = F.binary_cross_entropy_with_logits(pred, ground_truth)
          loss.backward()
          optimizer.step()
          total_loss += float(loss) * pred.numel()
          total_examples += pred.numel()
      print(f"Epoch: {epoch:03d}, Loss: {total_loss / total_examples:.4f}")
    # if epoch % 10 == 0:
    #     torch.save(model.state_dict(), f'./model_train/model_ep_{epoch}')


In [78]:
train_records = {u: set() for u in range(len(users))}
test_records = {u: set() for u in range(len(users))}

with open('./gnn_rel/train.txt', 'r') as f:
    lines = [l.strip() for l in f.readlines()]
    for line in lines:
        user_course = line.split('\t')

        user = user_course[0]
        course = user_course[1]

        train_records[int(user)].add(int(course))


with open('./gnn_rel/test.txt', 'r') as f:
    lines = [l.strip() for l in f.readlines()]
    for line in lines:
        user_course = line.split('\t')

        user = user_course[0]
        course = user_course[1]

        test_records[int(user)].add(int(course))

In [57]:
model.cpu()

Model(
  (user_emb): Embedding(6548, 16)
  (course_emb): Embedding(687, 16)
  (concept_emb): Embedding(3180, 16)
  (school_emb): Embedding(150, 16)
  (teacher_emb): Embedding(1683, 16)
  (gnn): GraphModule(
    (conv1): ModuleDict(
      (user__enroll__course): SAGEConv(16, 16, aggr=mean)
      (course__course_school__school): SAGEConv(16, 16, aggr=mean)
      (course__course_teacher__teacher): SAGEConv(16, 16, aggr=mean)
      (course__course_concept__concept): SAGEConv(16, 16, aggr=mean)
      (course__rev_enroll__user): SAGEConv(16, 16, aggr=mean)
      (school__rev_course_school__course): SAGEConv(16, 16, aggr=mean)
      (teacher__rev_course_teacher__course): SAGEConv(16, 16, aggr=mean)
      (concept__rev_course_concept__course): SAGEConv(16, 16, aggr=mean)
    )
    (conv2): ModuleDict(
      (user__enroll__course): SAGEConv(16, 16, aggr=mean)
      (course__course_school__school): SAGEConv(16, 16, aggr=mean)
      (course__course_teacher__teacher): SAGEConv(16, 16, aggr=mean)
 

In [79]:
all_enrolls = torch.tensor([[x,y] for x in range(len(users)) for y in range(len(courses))])
print(all_enrolls.shape)
all_enrolls = torch.transpose(all_enrolls, 1, 0)
all_enrolls[:, 20]

torch.Size([4498476, 2])


tensor([ 0, 20])

In [80]:
data_modif = data.clone()
data_modif['user', 'enroll', 'course']['edge_label_index'] = all_enrolls

In [48]:
data_modif

HeteroData(
  user={ node_id=[6548] },
  course={ node_id=[687] },
  school={ node_id=[150] },
  teacher={ node_id=[1683] },
  concept={ node_id=[3180] },
  (user, enroll, course)={
    edge_index=[2, 75597],
    edge_label_index=[2, 4498476],
  },
  (course, course_school, school)={ edge_index=[2, 687] },
  (course, course_teacher, teacher)={ edge_index=[2, 2284] },
  (course, course_concept, concept)={ edge_index=[2, 103497] },
  (course, rev_enroll, user)={ edge_index=[2, 75597] },
  (school, rev_course_school, course)={ edge_index=[2, 687] },
  (teacher, rev_course_teacher, course)={ edge_index=[2, 2284] },
  (concept, rev_course_concept, course)={ edge_index=[2, 103497] }
)

In [87]:
# Recommender - evaluation:

def evaluate():

  model.eval()

  item_embeds = model.course_emb
  user_embeds = model.user_emb

  item_set = set(data['course'].node_id.numpy())
  user_set = set(data['user'].node_id.numpy())

  k = 10
  scores = model(data_modif)

  precisions, recalls, ndcgs, hit = [], [], [], []
  for n, user in enumerate(user_set):
      test_item_list = list(item_set - train_records[user])
      item_score_map = {}

      user_scores = scores[n*len(courses):(n+1)*len(courses)]
      scores_test = user_scores[test_item_list]

      for i, s in zip(test_item_list, scores_test):
          item_score_map[i] = s.item()

      item_score_pair_sorted = sorted(item_score_map.items(), key=lambda x: x[1], reverse=True)
      item_sorted = [i[0] for i in item_score_pair_sorted]

      dcg = 0.0
      for i in item_sorted:
          if i in test_records[user]:
              dcg += 1. / (np.log(i + 2) / np.log(2))
      idcg = 0.0
      for i in range(min(len(item_sorted), len(test_records[user]))):
          idcg += 1. / (np.log(i + 2) / np.log(2))
      ndcg = dcg / idcg

      hit_num = len(set(item_sorted[:k]) & test_records[user])
      precisions.append(hit_num / k)
      recalls.append(hit_num / len(test_records[user]))
      ndcgs.append(ndcg)


  print(f'precision: {np.mean(precisions)}')
  print(f'recall: {np.mean(recalls)}')
  print(f'ndcg: {np.mean(ndcgs)}')


In [81]:
def getLoader(num_neg, num_neighbors):
  return LinkNeighborLoader(
    data=train_data,
    num_neighbors=num_neighbors,
    neg_sampling_ratio=num_neg,
    edge_label_index=(('user', 'enroll', 'course'), edge_label_index),
    edge_label=edge_label,
    batch_size=128,
    shuffle=True
)

In [82]:
neighbours_params = [{('course', 'course_school', 'school'):[2 ,2],
 ('course', 'course_teacher', 'teacher'):[2 ,2],
 ('course', 'course_concept', 'concept'):[7,7],
                   ('user', 'enroll', 'course'):[1,0],
                   ('course', 'rev_enroll', 'user'):[0,0],
 ('school', 'rev_course_school', 'course'):[2 ,2],
 ('teacher', 'rev_course_teacher', 'course'):[2 ,2],
 ('concept', 'rev_course_concept', 'course'):[7,7]},

{('course', 'course_school', 'school'):[2 ,2],
 ('course', 'course_teacher', 'teacher'):[2 ,2],
 ('course', 'course_concept', 'concept'):[10,10],
                   ('user', 'enroll', 'course'):[1,0],
                   ('course', 'rev_enroll', 'user'):[0,0],
 ('school', 'rev_course_school', 'course'):[2 ,2],
 ('teacher', 'rev_course_teacher', 'course'):[2 ,2],
 ('concept', 'rev_course_concept', 'course'):[10,10]},

  {('course', 'course_school', 'school'):[2 ,2],
 ('course', 'course_teacher', 'teacher'):[2 ,2],
 ('course', 'course_concept', 'concept'):[13,13],
('user', 'enroll', 'course'):[1,0],
('course', 'rev_enroll', 'user'):[0,0],
 ('school', 'rev_course_school', 'course'):[2 ,2],
 ('teacher', 'rev_course_teacher', 'course'):[2 ,2],
 ('concept', 'rev_course_concept', 'course'):[13,13]}]

In [ ]:
for embed_size in [16,32,64]:
  for m in [Model, ModelSimple]:
    for neigh in neighbours_params:
      for num_neg in [1.0,2.0,4.0]:
        for lr in [0.001, 0.005, 0.01]:
          train_loader = getLoader(num_neg, neigh)
          model = m(hidden_channels=embed_size)

          print(f'embed_size: {embed_size}, model: {m}, sampled_neighbours: {neigh}, num_negative_samples: {num_neg}, lr: {lr}')
          train(model, 10, lr)
          evaluate()


Proxy(x)
embed_size: 16, model: <class '__main__.Model'>, sampled_neighbours: {('course', 'course_school', 'school'): [2, 2], ('course', 'course_teacher', 'teacher'): [2, 2], ('course', 'course_concept', 'concept'): [7, 7], ('user', 'enroll', 'course'): [1, 0], ('course', 'rev_enroll', 'user'): [0, 0], ('school', 'rev_course_school', 'course'): [2, 2], ('teacher', 'rev_course_teacher', 'course'): [2, 2], ('concept', 'rev_course_concept', 'course'): [7, 7]}, num_negative_samples: 1.0, lr: 0.001


100%|██████████| 591/591 [00:12<00:00, 47.65it/s]


Epoch: 000, Loss: 0.5714


100%|██████████| 591/591 [00:12<00:00, 46.79it/s]


Epoch: 001, Loss: 0.4908


...

100%|██████████| 591/591 [00:14<00:00, 41.46it/s]


Epoch: 009, Loss: 0.4246
precision: 0.06405009163103238
recall: 0.20232511957154792
ndcg: 0.24016528656886124
Proxy(x)
embed_size: 32, model: <class '__main__.ModelSimple'>, sampled_neighbours: {('course', 'course_school', 'school'): [2, 2], ('course', 'course_teacher', 'teacher'): [2, 2], ('course', 'course_concept', 'concept'): [10, 10], ('user', 'enroll', 'course'): [1, 0], ('course', 'rev_enroll', 'user'): [0, 0], ('school', 'rev_course_school', 'course'): [2, 2], ('teacher', 'rev_course_teacher', 'course'): [2, 2], ('concept', 'rev_course_concept', 'course'): [10, 10]}, num_negative_samples: 2.0, lr: 0.005


100%|██████████| 591/591 [00:13<00:00, 45.42it/s]


Epoch: 000, Loss: 0.5495


 21%|██        | 123/591 [00:02<00:08, 53.05it/s]